# Verlet Integration

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Matrix Algebra

Warm up with matrix multiplication.

### Vector Manipulation

Topics: scalar and matrix multiplication, addition, inner products, outer products.

In [ ]:
# vectors u, v.
u = np.array([[1, 2]]).transpose()
v = np.array([[3, 4]]).transpose()

Some operations.

In [ ]:
print('vector addition: \n{}'.format(u + v))
print('vector transpose:\n{}'.format(u.transpose()))
print('vector inner product:\n{}'.format(u.transpose().dot(v)))
print('vector outer product:\n{}'.format(u.dot(v.transpose())))

### Matrix Manipulation

In [ ]:
# two 2x2 matrices.
A = np.array([
    [1, 2],
    [3, 4]
])
B = np.array([
    [2, 0],
    [0, 2]
])

Some operations.

In [ ]:
print('inverse matrix:\n{}'.format(np.linalg.inv(B)))
print('scalar matrix multiplication:\n{}'.format(2*A))
print('matrix addition:\n{}'.format(A + B))
print('matrix multiplication:\n{}'.format(A.dot(B)))
print('matrix vector multiplication:\n{}'.format(A.dot(u)))
print('inner product with matrices:\n{}'.format(u.transpose().dot(B.dot(v))))

## Verlet Method for a Two Body System

We write the solve function, which we'll use to find a solution using the verlet method.

In [ ]:
def solve(constructor, initial_condition, tt):
    # extract data from tt.
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    h = (t_max - t_min) / (n - 1)
    
    # method function.
    method_function = constructor(h)
    
    # create vector solution set.
    solution = [None for _ in range(n)]
    
    # compute solutions.
    for i, t in enumerate(tt):
        if i == 0:
            solution[0] = initial_condition
            pass
        else:
            solution[i] = method_function(t, solution[i-1])
            pass
        pass
    return solution

The verlet method for an n-body system is a system of three equations,

> $\mathbf{v}_{n+1/2} = \mathbf{v}_n + h/2\mathbf{M}^{-1}\mathbf{F}_n$,
>
> $\mathbf{q}_{n+1} = \mathbf{q}_n + h\mathbf{v}_{n+1/2}$
>
> $\mathbf{v}_{n+1} = \mathbf{v}_{n+1/2} + h/2\mathbf{M}^{-1}\mathbf{F}_{n+1}$

Where $\mathbf{F}_n = \mathbf{F}(\mathbf{q}_n) = -\nabla U(\mathbf{q}_n)$ is the gradient of the potential. We'll work with a simple 2-body system (variables $\mathbf{q}_1, \mathbf{q}_2, \mathbf{v}_1, \mathbf{v}_2$), where the potential energy is just the spring potential energy $U = \dfrac{1}{2}kr^2$, where $r^2 = (\mathbf{q}_1 - \mathbf{q}_2)^T\cdot(\mathbf{q}_1 - \mathbf{q}_2)$. Solving for the gradient gives $\nabla U = k(\mathbf{q}_1 - \mathbf{q}_2, \mathbf{q}_2 - \mathbf{q}_1)^T$.

In [ ]:
# potential function.
def grad_potential(q1, q2, k = 1):
    # q1 and q2 are vectors.
    return k*np.array([q1 - q2, q2 - q1])

In [ ]:
q1 = np.array([[1, 0]]).transpose()
q2 = np.array([[0, 2]]).transpose()

In [ ]:
grad_potential(q1, q2)

Unfortunately, the `dot` function isn't distributing through an array of two vectors properly, so we'll make a temporary `dist` function which does distribute some given matrix.

In [ ]:
def dist(m1, m2, two_vectors):
    return np.array([
        two_vectors[0]*(1/m1),
        two_vectors[1]*(1/m2)
    ])

In [ ]:
# just set mass vector to 1.
# assume system is two-dimensional.

def verlet_method_constructor_two_body(m1 = 1, m2 = 1):
    # option to specify masses.
    # solve for inverse mass matrix.
    
    def verlet_method_stepsize_two_body(h):
        def verlet_method_function_two_body(t, state):
            
            # extract data.
            q, v = state
            q1, q2 = q
            
            # calculate intermediate velocity.
            
            v_int = v + h/2*dist(m1, m2, -grad_potential(q1, q2))
            q_ = q + h*v_int
            
            # extract successor positions.
            q1_, q2_ = q_
            
            # calculate final velocity.
            v_ = v_int + h/2*dist(m1, m2, -grad_potential(q1_, q2_))
            
            # return new state.
            return np.array([q_, v_])
        return verlet_method_function_two_body
    return verlet_method_stepsize_two_body

In [ ]:
q1_init = np.array([[1, 0]]).transpose()
q2_init = np.array([[-1, 0]]).transpose()
v1_init = np.array([[0, 1]]).transpose()
v2_init = np.array([[0, 1]]).transpose()

constructor = verlet_method_constructor_two_body(m1 = 5)
initial_condition = np.array([[q1_init, q2_init], [v1_init, v2_init]])
tt = np.linspace(0, 10, 100)

In [ ]:
solution = solve(constructor, initial_condition, tt)

In [ ]:
qq = [s[0] for s in solution]
vv = [s[1] for s in solution]

q1 = [q[0] for q in qq]
q2 = [q[1] for q in qq]

q1x = [q[0] for q in q1]
q1y = [q[1] for q in q1]

q2x = [q[0] for q in q2]
q2y = [q[1] for q in q2]

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(q1x, q1y, label = 'a')
plt.plot(q2x, q2y, '--', label = 'b')
plt.legend()
plt.title('Two Body (Verlet)')
plt.show()

plt.plot(tt, np.array(q1x) - np.array(q2x))

## Verlet Generalization

However, we are not satisfied with just being able to simulate a two-body system with a spring potential: our goal is to be able to generalize this to an $n$-body system with as many dimensions and coordinates as possible, and with the freedom of being able to choose our potentials and respective masses.

In [ ]:
def matdist(O, vv):
    # O: a matrix.
    # vv: an array of vectors.
    
    if len(vv.shape) != 3:
        raise TypeError('Depth of second argument must be 3.')

    return np.array([O.dot(v) for v in vv])

Test with the identity.

In [ ]:
test_v1 = np.array([[1, 2]]).transpose()
test_v2 = np.array([[3, 5]]).transpose()
test_v3 = np.array([[1, 0]]).transpose()
vectors = np.array([test_v1, test_v2])
identity = np.array([
    [1, 0],
    [0, 1]
])

print(matdist(identity, vectors))

The function also works with covectors.

In [ ]:
print(matdist(test_v1.transpose(), vectors))

### Potential Energies

We will list a few types of potential energy functions which we can experiment with using our Verlet Integrator.

The total spring potential energy is

> $U = \sum\limits_{i = 1}^N\sum\limits_{j\neq i}\dfrac{1}{2}k(\mathbf{q}_i - \mathbf{q}_j)^2$,

and its gradient is

> $\dfrac{\partial U}{\partial\mathbf{q}_i} = \sum\limits_{j\neq i}k(\mathbf{q}_i - \mathbf{q}_j)$.

Similarly, the total gravitational potential energy is

> $U = -\sum\limits_{i = 1}^N\sum\limits_{j\neq i}\dfrac{1}{2}\dfrac{m_im_j}{r_{ij}}$

and its gradient is

> $\dfrac{\partial U}{\partial\mathbf{q}_i} = \sum\limits_{j\neq i}^N\dfrac{1}{2}\dfrac{m_im_j}{r_{ij}^2}$

In [ ]:
def grad_spring_potential_energy(state, k = 1, dim = 2):
    # state = [q, v]
    # extract data.
    
    q = state[0]
    n = len(q) # number of coordinates.
    
    # extract positions for each particle.
    qq = np.array([q[i:i+dim] for i in range(0, n, dim)])[0]
    m = len(qq)
    
    solution = [None for _ in range(m)]
    for i in range(m):
        jj = [list(filter(lambda j:j!= i, range(m)))]
        
        solution[i] = k* np.array([
            qq[i] - qq[j] for j in jj
        ]).sum(axis = 0)
        pass
    return solution

In [ ]:
q1 = np.array([[1, 2]]).transpose()
q2 = np.array([[1, 4]]).transpose()

v1 = np.array([[3, 4]]).transpose()
v2 = np.array([[-1, 0]]). transpose()

state = np.array([[q1, q2], [v1, v2]])

In [ ]:
grad_spring_potential_energy(state)